In [3]:
from disqco.circuits.cp_fraction import cp_fraction, cz_fraction
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from qiskit import transpile
from disqco.parti.FM.FM_methods import set_initial_partitions
from disqco.circuits.QAOA import QAOA_random
from qiskit.circuit.library import QFT
import pickle
from networkx import erdos_renyi_graph
from disqco.graphs.quantum_network import QuantumNetwork
from disqco.parti.FM.multilevel_FM import MLFM_recursive_hetero
import networkx as nx
import time

def produce_connected_graph(num_partitions, p):
    """
    Generates a connected graph with num_partitions nodes and edge probability p.
    """
    while True:
        graph = erdos_renyi_graph(num_partitions, p)
        if nx.is_connected(graph):
            return graph
        



num_qubits = 64
num_partitions = 8
qpu_sizes = [int(num_qubits/num_partitions)+1]*num_partitions


iterations = 100
all_costs = []
time_list = []
for k in range(iterations):

    circuit = cp_fraction(  num_qubits = num_qubits,
                            depth = num_qubits,
                            fraction = 0.5)

    # circuit = QAOA_random(num_qubits = num_qubits, prob=0.5, reps = 1)
    circuit = transpile(circuit, basis_gates=['u','cp'])

    graph = produce_connected_graph(num_partitions, 0.5)
    coupling = [[i,j] for i in range(num_partitions) for j in range(num_partitions) if i != j and graph.has_edge(i,j)]
    network = QuantumNetwork(qpu_sizes, coupling)
    hypergraph = QuantumCircuitHyperGraph(circuit)

    depth = hypergraph.depth
    initial_assignment = set_initial_partitions(network, num_qubits, depth)

    start = time.time()
    _, cost_list, _ = MLFM_recursive_hetero(graph = hypergraph,
                                            initial_assignment= initial_assignment,
                                            qpu_info= qpu_sizes,
                                            network= network,
                                            costs={},
                                            limit=num_qubits,
                                            stochastic=True,
                                            level_limit=None)
    end = time.time()
    print(f'Time taken: {end-start} seconds')
    time_list.append(end-start)
    
    print(f'Iteration {k+1}/{iterations} - Cost: {min(cost_list)}')
    all_costs.append(min(cost_list))
    





Time taken: 12.800840139389038 seconds
Iteration 1/100 - Cost: 737
Time taken: 13.05520510673523 seconds
Iteration 2/100 - Cost: 671
Time taken: 12.93991994857788 seconds
Iteration 3/100 - Cost: 668
Time taken: 13.026647090911865 seconds
Iteration 4/100 - Cost: 706
Time taken: 13.671057224273682 seconds
Iteration 5/100 - Cost: 645
Time taken: 13.506869077682495 seconds
Iteration 6/100 - Cost: 776
Time taken: 13.054991960525513 seconds
Iteration 7/100 - Cost: 731
Time taken: 13.852055072784424 seconds
Iteration 8/100 - Cost: 732
Time taken: 13.659689903259277 seconds
Iteration 9/100 - Cost: 690
Time taken: 13.888901948928833 seconds
Iteration 10/100 - Cost: 727
Time taken: 13.555052995681763 seconds
Iteration 11/100 - Cost: 611
Time taken: 13.185281991958618 seconds
Iteration 12/100 - Cost: 753
Time taken: 13.30055284500122 seconds
Iteration 13/100 - Cost: 664
Time taken: 13.40786099433899 seconds
Iteration 14/100 - Cost: 702
Time taken: 13.080195665359497 seconds
Iteration 15/100 - Cos

In [5]:
print(f'Average cost: {sum(all_costs)/len(all_costs)}')

Average cost: 697.67


In [7]:
with open('random_networks_MLFM-R_CP_0.5_n64_k8.dat', 'w') as f:
    f.write(f'x cost time \n')
    for i in range(iterations):
        f.write(f'{i} {all_costs[i]} {time_list[i]} \n')
    